In [4]:
# module dependencies
import os
import pandas as pd

# path to our data csv
f = os.path.join('Resources', 'purchase_data.csv')
# build a pandas dataframe from our csv
df = pd.read_csv(f)

## Player Count

In [5]:
# get all unique players
pcount = df['SN'].unique()
# count the unique players (could be done with .nunique() as well)
print(f'The number of unique players is {len(pcount)}.')

The number of unique players is 576.


## Purchasing Analysis (Total)

In [6]:
# number of unique items
uitems = len(df['Item ID'].unique())
# mean price
avg_price = df['Price'].mean()
# total purchase count
purchases = df['Purchase ID'].count()
# total revenue via sum on price
revenue = df['Price'].sum()
# put above values into dict for dataframe construction
d = {
    'Unique Items' : uitems,
    'Average Price' : '$' + str(round(avg_price, 2)),
    'Total Purchases' : purchases,
    'Total Revenue' : '$' + str(revenue)
    }
# build our dataframe
sum_df = pd.DataFrame(d, index = ['Value'])
sum_df

,Unique Items,Average Price,Total Purchases,Total Revenue
Value,183,$3.05,780,$2379.77


## Gender Demographics

In [7]:
# group by gender and count the number of unique vals in each group - pop to new dataframe
df2 = df.groupby("Gender")["SN"].nunique().to_frame()
df2

,SN
Gender,
Female,81
Male,484
Other / Non-Disclosed,11



## Purchasing Analysis (Gender)

In [8]:
# get total purchase values
dfa = df.groupby('Gender')['Price'].sum().tolist()
# get purchase counts
dfb = df.groupby('Gender')['Price'].count().tolist()
# get average purchase price
dfc = df.groupby('Gender')['Price'].mean().tolist()

# dict for our incoming dataframe
d = {
    'Index' : ['Female','Male','Other/Non-Disclosed'],
    'Total Purchase Value' : dfa,
    'Purchase Count' : dfb,
    'Average Purchase Price' : dfc,
}

# build our dataframe
ndf = pd.DataFrame({k: v for k, v in d.items() if not k == 'Index'},
                  index = d['Index'])

# calculate average purchase per person (by gender)
ndf['Average Total Purchase'] = ndf['Total Purchase Value'] / df.groupby('Gender')['SN'].nunique()

print(df.groupby('Gender')['SN'].nunique())
print('\n')

print(ndf)
print('\n')

# format our dataframe for $ signs
style_dict = {
    'Total Purchase Value' : '${0:,.2f}', 
    'Average Purchase Price' : '${0:,.2f}',
    'Average Total Purchase' : '${0:,.2f}'
}
ndf.style.format(style_dict)

Gender
Female                    81
Male                     484
Other / Non-Disclosed     11
Name: SN, dtype: int64


                     Total Purchase Value  Purchase Count  \
Female                             361.94             113   
Male                              1967.64             652   
Other/Non-Disclosed                 50.19              15   

                     Average Purchase Price  Average Total Purchase  
Female                             3.203009                4.468395  
Male                               3.017853                4.065372  
Other/Non-Disclosed                3.346000                     NaN  




,Total Purchase Value,Purchase Count,Average Purchase Price,Average Total Purchase
Female,$361.94,113,$3.20,$4.47
Male,"$1,967.64",652,$3.02,$4.07
Other/Non-Disclosed,$50.19,15,$3.35,$nan


## Age Demographics

In [9]:
bins = [0,9, 14, 19, 24, 29, 34, 39, 199]
labls = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

dfa = df
dfa['Age Range'] = pd.cut(df['Age'], bins, labels = labls)
dfa = dfa.groupby('Age Range').nunique()
dfa = dfa.drop(['Purchase ID', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'Age Range'], axis=1)
dfa.rename(columns = {'SN' : 'Total Count'}, inplace = True)
s = dfa['Total Count'].sum()
dfa['Percentage of Players'] = dfa['Total Count'] / s
style_dict = { 'Percentage of Players' : '{:.2%}' }
dfa.style.format(style_dict)


,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [10]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 199]
labls = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

dfa = df
dfa['Age Range'] = pd.cut(df['Age'], bins, labels = labls)
dff = pd.DataFrame()

dff['PurchaseCount'] = dfa.groupby('Age Range').SN.count()
dff['TotalPurchaseValue'] = dfa.groupby('Age Range').Price.sum()
dff['AvgPurchasePrice'] = dfa.groupby('Age Range').Price.mean()
dff['AvgTotalPurchasePerPlayer'] = dff['TotalPurchaseValue'] / dfa.groupby('Age Range').SN.nunique()

# format our dataframe for $ signs
style_dict = {
    'TotalPurchaseValue' : '${0:,.2f}', 
    'AvgPurchasePrice' : '${0:,.2f}',
    'AvgTotalPurchasePerPlayer' : '${0:,.2f}'
}
dff.style.format(style_dict)

,PurchaseCount,TotalPurchaseValue,AvgPurchasePrice,AvgTotalPurchasePerPlayer
Age Range,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,"$1,114.06",$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
40+,13,$38.24,$2.94,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [30]:
b = pd.DataFrame()
b['Spent'] = df.groupby('SN')['Price'].sum()
b['Purchases'] = df.groupby('SN')['Price'].count()
b['Average Expenditure'] = b['Spent'] / b['Purchases']
c = b.sort_values(by = 'Spent', ascending = False)
c.head()

,Spent,Purchases,Average Expenditure
SN,,,
Lisosia93,18.96,5,3.792000
Idastidru52,15.45,4,3.862500
Chamjask73,13.83,3,4.610000
Iral74,13.62,4,3.405000
Iskadarya95,13.10,3,4.366667


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [65]:
a = df[['Item ID','Item Name','Price']]

b = pd.DataFrame()
#b['Price'] = a.groupby(['Item ID','Item Name'])['Price']
b['Count'] = a.groupby(['Item ID','Item Name'])['Price'].count()
b['Total'] = a.groupby(['Item ID','Item Name'])['Price'].sum()
b['Price'] = b['Total'] / b['Count']

c = b[['Price','Count','Total']]
d = c.sort_values(by = 'Count', ascending = False)
d

,,Price,Count,Total
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,12,50.76
145,Fiery Glass Crusader,4.58,9,41.22
108,"Extraction, Quickblade Of Trembling Hands",3.53,9,31.77
82,Nirvana,4.90,9,44.10
19,"Pursuit, Cudgel of Necromancy",1.02,8,8.16
...,...,...,...,...
104,Gladiator's Glaive,1.93,1,1.93
23,Crucifer,1.99,1,1.99
180,Stormcaller,3.36,1,3.36


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [66]:
d = c.sort_values(by = 'Total', ascending = False)
d

,,Price,Count,Total
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,12,50.76
82,Nirvana,4.90,9,44.10
145,Fiery Glass Crusader,4.58,9,41.22
92,Final Critic,4.88,8,39.04
103,Singed Scalpel,4.35,8,34.80
...,...,...,...,...
125,Whistling Mithril Warblade,1.00,2,2.00
126,Exiled Mithril Longsword,2.00,1,2.00
23,Crucifer,1.99,1,1.99
